### Os, Sync, Env

In [2]:
import os
import dotenv
dotenv.load_dotenv(dotenv_path="../.env")
import nest_asyncio
nest_asyncio.apply()

### LLM & EMBEDDING MODEL

In [3]:
# GROQ
QROQ_API_KEY = os.environ['GROQ_API_KEY_1']
from llama_index.llms.groq import Groq
groq_llm = Groq(model="llama3-8b-8192", api_key = QROQ_API_KEY, set_run_config=None)
print("Groq: "+str(groq_llm.complete("Hi")))
# Cohere
CO_API_KEY = os.environ['COHERE_API_KEY']
from llama_index.llms.cohere import Cohere
cohere_llm = Cohere(model="command-light",api_key=CO_API_KEY)
print("Cohere: "+str(cohere_llm.complete("Hi")))
# Fast Embed
from llama_index.embeddings.fastembed import FastEmbedEmbedding
fast_embed = FastEmbedEmbedding(model_name="BAAI/bge-base-en-v1.5")

d:\Learning New\GenAI\Project_RAG\RAG\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


Groq: Hi! It's nice to meet you. Is there something I can help you with or would you like to chat?
Cohere:  Hi! Good day! I am ready to help and happy to talk to you? how are you doing?  I can assist you with


Fetching 5 files: 100%|██████████| 5/5 [00:00<?, ?it/s]


### SETTINGS

In [4]:
from llama_index.core.settings import Settings
Settings.llm = groq_llm
Settings.embed_model = fast_embed

### Loading Index from Persisted Index Stores

In [5]:
from llama_index.core import StorageContext, load_index_from_storage

storage_context1 = StorageContext.from_defaults(persist_dir="D:/Learning New/GenAI/Project_RAG/RAG/1. Data/index_store")
index_from_docs = load_index_from_storage(storage_context1)
storage_context2 = StorageContext.from_defaults(persist_dir="D:/Learning New/GenAI/Project_RAG/RAG/1. Data/index_from_node_store")
index_from_nodes = load_index_from_storage(storage_context2)

### Query

In [10]:
query = "Difference - PEB241T and PEB347T. What is capacity for each instrument?"

### Retrieval (Vector)

In [7]:
retriever_from_docs = index_from_docs.as_retriever(
    similarity_top_k=10,
    similarity_threshold=0.75)
retriever_from_nodes = index_from_nodes.as_retriever(
    similarity_top_k=10,
    similarity_threshold=0.75)

### Query Engine (Vector)

In [ ]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# configure a post processor
similarity_processor = SimilarityPostprocessor(similarity_cutoff=0.42)

# configure a response sythesizer
response_synthsizer = get_response_synthesizer(llm=Settings.llm)

# create a query engine
default_query_engine_from_docs = RetrieverQueryEngine(
    retriever=retriever_from_docs,
    response_synthesizer=response_synthsizer,
    node_postprocessors=[similarity_processor],
)
# create a query engine
default_query_engine_from_nodes = RetrieverQueryEngine(
    retriever=retriever_from_nodes,
    response_synthesizer=response_synthsizer,
    node_postprocessors=[similarity_processor],
)

According to the context information, the capacity for PEB241T is 220g and 320g, and for PEB347T, it is not specified.


Response(response='The capacity for PEB241T is 220g and 320g, and the capacity for PEB347T is also 220g and 320g.', source_nodes=[NodeWithScore(node=TextNode(id_='3c8c90b6-42ec-41e3-96cc-83d240174045', embedding=None, metadata={'Instrument type': 'Precision Electronic Balance', 'Sub category': '1mg', 'Model': 'PEB241T, PEB347T'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='429d4a3c-e10b-4391-805a-c5ae0eac5442', node_type='4', metadata={'Instrument type': 'Precision Electronic Balance', 'Sub category': '1mg', 'Model': 'PEB241T, PEB347T'}, hash='248a40a4e0bfd882a6976a1f436985303bedd8ba96ac4b3cb0cee7b131ddfc3f'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='0ae4b4c4-74a5-4f52-b814-cc6a7fc96ee3', node_type='1', metadata={'Instrument type': 'Precision Electronic Balance', 'Sub category': '1mg', 'Model': 'PEB241T, PEB347T'}, hash='1df79e8760083db405a7e2e226cea5e574dc2ac3a9b29f8f6736b87ad

### Querying (Vector)

In [13]:
# Querying
print(default_query_engine_from_docs.query(query).response)
print(default_query_engine_from_nodes.query(query).response)

According to the context information, the capacity for PEB241T is 220g and 320g, and for PEB347T, it is not mentioned.
The capacity for PEB241T is 220g and 320g, and the capacity for PEB347T is also 220g and 320g.


### Retrieval (BM25)

In [8]:
from llama_index.retrievers.bm25 import BM25Retriever

bm25_retriever_from_nodes = BM25Retriever.from_defaults(docstore=index_from_nodes.docstore, similarity_top_k=10)
bm25_retriever_from_docs = BM25Retriever.from_defaults(docstore=index_from_docs.docstore, similarity_top_k=10)

resource module not available on Windows


### Query Engine (BM25)

In [14]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# configure a post processor
similarity_processor = SimilarityPostprocessor(similarity_cutoff=0.42)

# configure a response sythesizer
response_synthsizer = get_response_synthesizer(llm=Settings.llm)

# create a query engine
bm25_query_engine_from_docs = RetrieverQueryEngine(
    retriever=bm25_retriever_from_docs,
    response_synthesizer=response_synthsizer,
    node_postprocessors=[similarity_processor],
)
# create a query engine
bm25_query_engine_from_nodes = RetrieverQueryEngine(
    retriever=bm25_retriever_from_nodes,
    response_synthesizer=response_synthsizer,
    node_postprocessors=[similarity_processor],
)

### Querying (BM25)

In [15]:
# Querying
print(bm25_query_engine_from_docs.query(query).response)
print(bm25_query_engine_from_nodes.query(query).response)

The capacity for PEB241T is 220g and the capacity for PEB347T is 320g.
The capacity for PEB241T is 220g and 320g, and for PEB347T it is also 220g and 320g.


### Retrieval (Hybrid)

In [23]:
from llama_index.core.retrievers import QueryFusionRetriever

hybrid_retriever_from_docs = QueryFusionRetriever(
    [bm25_query_engine_from_docs, retriever_from_docs],
    similarity_top_k=10,
    num_queries=1,
    mode="reciprocal_rerank",
    use_async=True,
    verbose=True,
)
hybrid_retriever_from_nodes = QueryFusionRetriever(
    [bm25_query_engine_from_nodes, retriever_from_nodes],
    similarity_top_k=10,
    mode="reciprocal_rerank",
    num_queries=1,
    use_async=True,
    verbose=True,
)

### Query Engine (Hybrid)

In [24]:
from llama_index.core import get_response_synthesizer
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# configure a post processor
similarity_processor = SimilarityPostprocessor(similarity_cutoff=0.02)

# configure a response sythesizer
response_synthsizer = get_response_synthesizer(llm=Settings.llm)

# create a query engine
hybrid_query_engine_from_docs = RetrieverQueryEngine(
    retriever=hybrid_retriever_from_docs,
    response_synthesizer=response_synthsizer,
    node_postprocessors=[similarity_processor],
)
# create a query engine
hybrid_query_engine_from_nodes = RetrieverQueryEngine(
    retriever=hybrid_retriever_from_nodes,
    response_synthesizer=response_synthsizer,
    node_postprocessors=[similarity_processor],
)

### Querying (Hybrid)

In [25]:
# Querying
print(hybrid_query_engine_from_docs.query(query).response)
print(hybrid_query_engine_from_nodes.query(query).response)

The capacity for PEB241T is 220g and the capacity for PEB347T is 320g.
The capacity for PEB241T is 220g and the capacity for PEB347T is 320g.
